<a href="https://colab.research.google.com/github/NosenkoArtem/Categorical-Encoding/blob/master/%D0%94%D0%97_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 2

## Задача 1. Неявный метод оценивания score-функции

На лекции мы обсудили методолгоию score matching, где функция потерь для обучения градиента логарифма плотности данных представляется как
$$
    \frac{1}{2} \mathbb{E}_{\pi}\bigl\| \mathbf{s}_{\boldsymbol{\theta}}(\mathbf{x}) - \nabla_\mathbf{x} \log \pi(\mathbf{x}) \bigr\|^2_2 \rightarrow \min_{\boldsymbol{\theta}}.
$$


В частности, мы познакомились с выводом **implicit score matching**.

Напомним, что функция потерь для обучения  **implicit score matching** выражается так:
$$
\frac{1}{2} \mathbb{E}_{\pi}\bigl\| s_{\boldsymbol{\theta}}(x) - \nabla_x \log \pi(x) \bigr\|^2_2 = \mathbb{E}_{\pi}\left[ \frac{1}{2}s^2_{\boldsymbol{\theta}}(x) + \nabla_{x} s_{\boldsymbol{\theta}}(x) \right] + \text{const}.
$$

В данном задании вам необходимо ответить на следующие вопросы:

- Почему выражение справа в последней формуле с точки зрения практики лучше для нас, чем выражение слева?  

- По каким причинам мы не пользуемся данной методологией **implicit score matching** и обращаемся к **denoising score matching** ?

- Повторите вывод из лекции для **implicit score matching** в одномерном случае.

In [ ]:
# your latex code is here

## Задача 2. Марковские цепи

Пусть последовательность $X_{0},...,X_{T}$ образует марковскую цепь. Покажите, что обратная по времени последовательность $X_{T},...,X_{0}$ тоже является марковской цепью, то есть:

$$ p_{X_{t-1}|X_{t},...,X_{T}}(x_{t-1}|x_{t},...,x_{T}) = p_{X_{t-1}|X_{t}}(x_{t-1}|x_{t}) .$$

In [ ]:
# your latex code is here

## Задача 3. Зашумленный метод оценивания score-функции для двумерных данных

В данной задаче вам предстоит дописать недостающие элементы в коде. Стоит отметить, что данная реализация не совсем такая, как в семинаре, и сделано это специально, чтобы вы не просто взяли код из семинара, а смогли и сами написать все необходимые блоки.

Запускать данный метод **denoising score matching** будем  на датасете лун, который выглядит так, как на картинке ниже.

Тогда давайте напишем функцию, которая создает такой датасет.

![title](https://i2.wp.com/miro.medium.com/1*itR36uLRt36bmFQVh-cVtQ.jpeg)

Сначала импортируем необходимые библиотеки. А уже потом заводим функцию **generate_moons_data**, которая возвращает нам данные, располагающиеся в виде лун.

In [ ]:
from tqdm import tqdm
from typing import List, Tuple
import math

import numpy as np
from sklearn.datasets import make_moons

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

In [ ]:
def generate_moons_data(count: int) -> tuple:
    data, labels = make_moons(n_samples=count, noise=0.1)
    data = data.astype("float32")
    split = int(0.8 * count)
    train_data, test_data = data[:split], data[split:]
    train_labels, test_labels = labels[:split], labels[split:]
    return train_data, train_labels, test_data, test_labels

Возьмем 5000 точек для визуализации данных:

In [ ]:
COUNT = 5000

train_data, train_labels, test_data, test_labels = generate_moons_data(COUNT)

Давайте вспомним теорию.  


Идея заключается в следующем. Мы определяем score-функцию
$$
    \mathbf{s}_{\boldsymbol{\theta}}(\mathbf{x}) = \nabla_{\mathbf{x}}\log p(\mathbf{x}| \boldsymbol{\theta}).
$$

Наша основная задача состоит в том, что нам хотелось бы минимизировать дивергенцию Фишера, чтобы получить score-функцию:
$$
    D_F(\pi, p) = \frac{1}{2}\mathbb{E}_{\pi}\bigl\| \mathbf{s}_{\boldsymbol{\theta}}(\mathbf{x}) - \nabla_{\mathbf{x}} \log \pi(\mathbf{x}) \bigr\|^2_2 \rightarrow \min_{\boldsymbol{\theta}}.
$$

И если нам бы удалось оценить score-функцию, то мы могли бы использовать динамику Ланжевена для генерации выборки из необходимого распределения:
$$
    \mathbf{x}_{l + 1} = \mathbf{x}_l + \frac{\eta}{2} \cdot \nabla_{\mathbf{x}_l} \log p(\mathbf{x}_l | \boldsymbol{\theta}) + \sqrt{\eta} \cdot \boldsymbol{\epsilon}, \quad \boldsymbol{\epsilon} \sim \mathcal{N}(0, \mathbf{I}).
$$

Однако дивергенция Фишера трудноразрешима (так как мы не знаем честную score-функцию), и мы используем процедуру добавления шума **denoising score matching**  для оценки score-функции по зашумленным данным $$\mathbf {x}_{\sigma} = \mathbf {x} + \sigma \cdot \boldsymbol{\epsilon}.$$

Как мы выяснили на лекции, минимизация дивергенции Фишера для зашумленных данных эквивалентна следующей задаче:
\begin{multline*}
    \mathbb{E}_{q(\mathbf{x}_{\sigma})}\bigl\| \mathbf{s}_{\boldsymbol{\theta}, \sigma}(\mathbf{x}_{\sigma}) - \nabla_{\mathbf{x}_{\sigma}} \log q(\mathbf{x}_{\sigma}) \bigr\|^2_2
    = \mathbb{E}_{\pi(\mathbf{x})} \mathbb{E}_{q(\mathbf{x}_{\sigma} | \mathbf{x})}\bigl\| \mathbf{s}_{\boldsymbol{\theta}, \sigma}(\mathbf{x}_{\sigma}) - \nabla_{\mathbf{x}_{\sigma}} \log q(\mathbf{x}_{\sigma} | \mathbf{x}) \bigr\|^2_2 + \text{const}(\boldsymbol{\theta}).
\end{multline*}

Здесь
$$
    \log q(\mathbf{x}_{\sigma} | \mathbf{x}) = - \frac{\mathbf{x}_{\sigma} - \mathbf{x}}{\sigma^2} = - \frac{\boldsymbol{\epsilon}}{\sigma}.
$$

Тогда финальная формула для denoising score matching это

$$
\mathbb{E}_{\pi(\mathbf{x})} \mathbb{E}_{q(\mathbf{x}_{\sigma} | \mathbf{x})}\bigl\| \mathbf{s}_{\boldsymbol{\theta}, \sigma}(\mathbf{x}_{\sigma}) + \frac{\boldsymbol{\epsilon}}{\sigma} \bigr\|^2_2 \rightarrow \min_{\boldsymbol{\theta}}.
$$

In [ ]:
class DenoisingScoreMatcher(nn.Module):
    def __init__(
            self,
            score_model: nn.Module,
            input_shape: Tuple[int],
            sigma: float
        ):
        super().__init__()

        self.score_model = score_model
        self.input_shape = input_shape
        self.sigma = sigma

    @property
    def device(self):
        return next(self.parameters()).device

    def forward(self, x: torch.Tensor):
        # ====
        # your code
        # написать, как семплируется гауссовский шум
        # получить зашумленные данные через исходные и шум
        noisy_x =
        # =====

        # посчитать score-функцию для зашумленных данных
        s = self.score_model(noisy_x)

        # ====
        # your code
        # запишите функцию потерь метода
        # это МСЕ Loss

        # =====
        return loss

    def loss(self, x: torch.Tensor):
        return {"total_loss": self(x).mean(dim=0).sum()}

    def langevin_dynamics(self, x: torch.Tensor, num_steps: int, eta: float):
        # =====
        # your code
        # Запишите итеративную процедуру динамики Ланжевена

        # =====
        return x

    def sample(self, num_samples: int = 64, num_steps: int=100, eta: float = 0.01):
        with torch.no_grad():
            # мы семплируем x_0 из равномерного распределения на отрезке U[-1, 1]
            x0 = 2. * torch.rand_like(torch.empty(num_samples, *self.input_shape)) - 1.
            x0 = x0.to(self.device)

            # запустить динамику Ланжевена
            x = self.langevin_dynamics(x0, num_steps=num_steps, eta=eta)
        return x


def test_denoiser_score_matcher():
    matcher = DenoisingScoreMatcher(
        score_model=nn.Linear(2, 2),
        input_shape=(2,),
        sigma=0.1
    )
    x = torch.rand(16, 2)
    assert x.size() == matcher(x).size()
    loss = matcher.loss(x)["total_loss"]
    assert len(loss.size()) == 0
    assert list(matcher.sample(4).size()) == [4, 2]


test_denoiser_score_matcher()

In [ ]:
# ====
# your code
# Выберите гиперпараметры метода

BATCH_SIZE =
EPOCHS =   # > 50
LR =   # > 1e-3
HIDDEN_SIZE =   # > 32
SIGMA =  # 0.01 < x < 1.0
# ====

train_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

# ====
# your code
# определите архитектуру модели, смотрите семинар 2
#
score_model =
# ====

matcher = DenoisingScoreMatcher(
    score_model=score_model, input_shape=(2,), sigma=SIGMA
)


optimizer = torch.optim.Adam(matcher.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)




Также вам необходимо описать функцию тренировки данной модели и запустить ее.

In [ ]:


def train_model(
    matcher,
    train_loader,
    test_loader,
    epochs=EPOCHS,
    optimizer=optimizer,
    scheduler=scheduler,
    device=DEVICE,
    n_samples=512):


  # и теперь, определив саму модель, необходимо написать самый базовый торчовский цикл обучения модели




Уже обученную модель matcher можно запустить и посмотреть, какие семплы она воспроизводит.

In [ ]:
# ====
# your code
# выберите параметры для семплирования динамикой Ланжевена
NUM_STEPS =
ETA =
# ====

samples = matcher.sample(num_samples=5000, num_steps=NUM_STEPS, eta=ETA).cpu()

Опишите полученные вами изображения.